In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

LOCAL_FOLDER = "G:/notebook/MNIST_data/"
np.random.seed(10)
tf.set_random_seed(10)

data = input_data.read_data_sets(LOCAL_FOLDER, one_hot=True, validation_size=0)

Extracting G:/notebook/MNIST_data/train-images-idx3-ubyte.gz
Extracting G:/notebook/MNIST_data/train-labels-idx1-ubyte.gz
Extracting G:/notebook/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting G:/notebook/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def dense_layer(x, in_dim, out_dim, layer_name, act):
    with tf.name_scope(layer_name):
        # W
        weights = tf.Variable(
            tf.random_uniform(
                [in_dim, out_dim], 
                maxval = tf.sqrt(6.0) / tf.sqrt(float(out_dim+in_dim)),
                seed = 10
            ), name="weights"
        )

        # b
        biases = tf.Variable(tf.zeros([out_dim]), name="biases")

        # y =f(Wx+b)
        layer = act(tf.matmul(x, weights) + biases, name="activations")

    return layer

In [3]:
input = tf.placeholder(tf.float32, [None, 784], name="input")
targets = tf.placeholder(tf.float32, [None, 10], name="targets")

# network layers: two hidden and one output
hidden1 = dense_layer(input, 784, 200, "hidden1", act=tf.nn.relu)
#hidden1 = dense_layer(input, 784, 200, "hidden1", act=tf.identity)
#hidden2 = dense_layer(hidden1, 200, 300, "hidden2", act=tf.nn.relu)
output = dense_layer(hidden1, 200, 10, "output", act=tf.identity)


# loss function: cross-entropy with built-in 
# (stable) computation of softmax from logits
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=targets, logits=output
    )
)


# training algorithm: Adam with configurable learning rate
#train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)
#train_step = tf.train.MomentumOptimizer(0.01, 0.9).minimize(cross_entropy)
#train_step = tf.train.AdagradOptimizer(0.1).minimize(cross_entropy)
#train_step = tf.train.RMSPropOptimizer(0.1).minimize(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
#train_step = tf.train.AdadeltaOptimizer(1.0).minimize(cross_entropy)

# evaluation operation: ratio of correct predictions
correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

AttributeError: 'Tensor' object has no attribute 'weights'

In [27]:
from matplotlib import pyplot
import numpy as np

steps = []
accuracies = []

# creating session
sess = tf.InteractiveSession()

# initializing trainable variables
sess.run(tf.global_variables_initializer())

# training loop
for step in range(4000):
    # fetching next batch of training data
    batch_xs, batch_ys = data.train.next_batch(100)

    if step % 100 == 0:
        # reporting current accuracy of the model on every 100th batch
        batch_accuracy = sess.run(accuracy, feed_dict={input: batch_xs, targets: batch_ys})
        #print("{0}:\tbatch accuracy {1:.2f}".format(step, batch_accuracy))
        steps.append(step/100)
        accuracies.append(batch_accuracy)

    # running the training step with the fetched batch
    sess.run(train_step, feed_dict={input: batch_xs, targets: batch_ys})

pyplot.plot(steps, accuracies, 'k-', linewidth=2.0, color='green')
pyplot.show()    

In [28]:
# evaluating model prediction accuracy of the model on the test set
test_accuracy = sess.run(accuracy, feed_dict={input: data.test.images, targets: data.test.labels})


print("-------------------------------------------------")
print("Test set accuracy: {0:.4f}".format(test_accuracy))

-------------------------------------------------
Test set accuracy: 0.9574
